# Project Introduction:
Primer laboratorio de aprendizaje automático, desarrollado por Sergio Barragán Blanco (100472343) y Eduardo Alarcón Navarro (100472175). 
Grupo 17.


# 1. EDA
Existen 23 carácterísticas que definen cada momento, de las cuales ninguna es categórica, todas son numéricas (con la energía suman 24). No existen valores faltantes, pero si los hubiera, los rellenaríamos con la media del valor superior e inferior antes de randomizar el dataset.

No existen tampoco columnas constantes, que se eliminarían. 

Con todo esto, podemos observar que es un problema de regresión.

La variable que estamos intentando predecir es la "energía" que es el valor de la enrgía generada 24 horas después. 

## Correlaciones entre parámetros:
Al principio, parece que las columnas lai_lv.13 y lai_hv.13 tienen una correlación, pero según avanza el tiempo, desaparece.

## Escala de los datos
Entre las diferentes columnas de datos, tenemos valores y magnitudes muy dispares.

In [5]:
import pandas as pd

# Load the data
data = pd.read_csv('wind_ava.csv.gz', compression='gzip')
# FIlter the data to only include the columns that end in 13

data = data.filter(regex='13$|energy|datetime')


print(data)


                 datetime   energy    p54.162.13  p55.162.13    cape.13  \
0     2005-01-02 18:00:00   402.71  2.510824e+06    9.186295  13.527577   
1     2005-01-03 00:00:00   696.80  2.513173e+06    8.849569   6.896412   
2     2005-01-03 06:00:00  1591.15  2.509627e+06    7.924080   4.774439   
3     2005-01-03 12:00:00  1338.62  2.510571e+06    6.922709   0.000000   
4     2005-01-03 18:00:00   562.50  2.505664e+06    6.646282   0.000000   
...                   ...      ...           ...         ...        ...   
4743  2009-12-30 06:00:00  1280.13  2.385772e+06   14.413342  24.971074   
4744  2009-12-30 12:00:00   855.00  2.385708e+06   12.802883  60.248874   
4745  2009-12-30 18:00:00   117.06  2.379049e+06   10.915997   3.145067   
4746  2009-12-31 12:00:00   516.96  2.371697e+06   13.048144  12.580268   
4747  2009-12-31 18:00:00   867.07  2.369547e+06    9.172745   8.487892   

        p59.162.13  lai_lv.13  lai_hv.13   u10n.13   v10n.13  ...      t2m.13  \
0     1.386937e+06